## data preprocessing and post classification analysis

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter as C
import re

In [2]:
all_data = pd.read_csv("../data/data/all_data.csv")

In [ ]:
## all files in models ie logs
#[os.path.join(top, file) for top, dirs, files in os.walk("./hate-speech-detection/models") for file in files]

In [ ]:
## all images
#os.listdir("../data/data/img")

In [ ]:
## displaying all rows of df
#pd.set_option('display.max_rows', None)

In [ ]:
# getting test data
#test_data = pd.read_json("../data/data/test.jsonl", lines=True)
#test = test_data.sort_values(by=["img"]).reset_index(drop=True)

In [3]:
def open_accuracys(path):
    # opening .accuracys files and getting the file names of the images with the lowest and highest squared error
    with open(path) as f:
        file = f.read()
        file = file.split("\n")
    file = [x.split("\t")[0].split("/")[5].split(".")[0] for x in file if x.startswith("..")]
    return file[:20], file[20:] # lowest squared error, highest squared error

In [ ]:
top100_0_11, bott100_0_11 = open_accuracys("./hate-speech-detection/results/models/clf100.0.11.pt.accuracys")

In [ ]:
top100_0_b11, bott100_0_b11 = open_accuracys("./hate-speech-detection/results/models/clf100.0.b11.pt.accuracys")

In [ ]:
top3b01, bott3b01 = open_accuracys("./hate-speech-detection/results/accuracys")

In [ ]:
top_bott = [top100_0_11, bott100_0_11, top100_0_b11, bott100_0_b11, top3b01, bott3b01]

In [ ]:
from PIL import Image
for x in top100_0_b11:
    im = Image.open("../data/data/img/" + str(x) + ".png")
    im.show()

In [ ]:
def flatten_list(lst):
    return [subsublist for sublist in lst for subsublist in sublist]

In [ ]:
def get_text_counts(lst):
    
    x = all_data[all_data["id"].isin(lst)]
    x = x.text.to_list()
    
    # getting a list of bigrams for each sentence (including punctuation unfortunately) by splitting on everything that's not a word
    # maybe rm punctuation instead?
    bigrams = [{(w.lower(), re.split(r"(\W+)", sent)[i+1].lower()) for i, w in enumerate(sent.split(" ")) if i != (len(sent.split(" "))-1)} for sent in x ]
    bigrams = flatten_list(bigrams)
    
    words = [{w.lower() for w in set(re.split(r"(\W+)", sent))} for sent in x]
    words = flatten_list(words)
    
    count_words = C(words)
    count_bigrams = C(bigrams)
    
    return dict(sorted(count_words.items(), key=lambda item: item[1], reverse=True)), dict(sorted(count_bigrams.items(), key=lambda item: item[1], reverse=True))

In [ ]:
top100_0_11c, bott100_0_11c, top100_0_b11c, bott100_0_b11c, top3b01c, bott3b01c = [get_text_counts(l) for l in top_bott]

In [ ]:
top3b01c[0]

In [ ]:
def find_words(w_list, img_list):
    x = all_data[all_data["id"].isin(img_list)].reset_index(drop=True)
    w_id = {}
    for row in x.iterrows():
        split = re.split(r"(\W+)", row[1].text)
        img = row[1].id
        for w in split:
            if w in w_list:
                if w not in w_id.keys():
                    w_id[w] = set()
                    w_id[w].add(img)
                else:
                    w_id[w].add(img)
    return w_id
    

In [ ]:
find_words(["muslim", "muslims", "islam", "religion"], top100_0_11)

In [ ]:

top_accs_tok[0].intersection(top_accs_tok[1], top_accs_tok[2], top_accs_tok[3], top_accs_tok[4], top_accs_tok[5])

In [ ]:
set.intersection(*[set(x) for x in top_accs_tok])  

In [ ]:
top_accs_tok

In [ ]:
#origin = "../data/data/"
#destination = "../data/data/test/"
#os.mkdir("../data/data/test/")
#for img in test_data.img.tolist():
#    os.rename((origin+img), (destination+img.lstrip("img/")))
#    

In [ ]:
dev_data = pd.read_json("../data/data/dev.jsonl", lines=True)

In [ ]:
train_data = pd.read_json("../data/data/train.jsonl", lines=True)

In [ ]:
all_data = pd.concat([dev_data, train_data], ignore_index=True)

In [ ]:
# len label 0: 5700 = 0.63
# len label 1: 3300 = 0.37

In [ ]:
all_data[all_data["label"] == 0]

In [ ]:
all_data['id'] = list(map(lambda x: str(x).zfill(5), all_data['id']))

In [ ]:
all_data['set'] = all_data.shape[0] * ['train']

In [ ]:
testsize = int(((15/100))*all_data.shape[0])
valsize = testsize

In [ ]:
testsize

In [ ]:
test = testsize * ['test']

In [ ]:
val = valsize * ['val']

In [ ]:
len(test)

In [ ]:
#df = all_data

In [ ]:
all_data = all_data.sample(frac=1).reset_index(drop=True) # shuffle

In [ ]:
all_data.loc[:(testsize-1), 'set'] = test # 

In [ ]:
all_data.loc[testsize:(testsize+valsize-1), 'set'] = val

In [ ]:
all_data.to_csv("../data/data/all_data.csv", index=False)


In [ ]:
all_data = df

In [ ]:
df = pd.read_csv("../data/data/all_data.csv")

In [ ]:
df.shape

In [ ]:
not_val = df[(df['set'] == 'val') & (df['label'] == 0)].reset_index(drop=True)
off_val = df[(df['set'] == 'val') & (df['label'] == 1)].reset_index(drop=True)

In [ ]:
not_train = df[(df['set'] == 'train') & (df['label'] == 0)].reset_index(drop=True)
off_train = df[(df['set'] == 'train') & (df['label'] == 1)].reset_index(drop=True)

In [ ]:
not_train.shape[0]/(not_train.shape[0] + off_train.shape[0])

In [ ]:
not_test = df[(df['set'] == 'test') & (df['label'] == 0)].reset_index(drop=True)
off_test = df[(df['set'] == 'test') & (df['label'] == 1)].reset_index(drop=True)

In [ ]:
pd.concat([not_val, off_val, not_test, off_test, not_train, off_train])

In [ ]:
856/(856+494)

In [ ]:
not_train_size = not_train[:2304]

In [ ]:
3996+2304

In [ ]:
3996/(3996+2304)

In [ ]:

if test:
    print("hello")

In [ ]:
def write_txt(df, filename):
    file = "../data/data/" + filename
    with open(file, 'w') as f:
        for img in df.img.to_list():
            f.write(img+"\n")
    print("Done!")
        

In [ ]:
write_txt(not_test, "goodMemesList.txt.test")
write_txt(off_test, "hateMemesList.txt.test")
write_txt(not_train, "goodMemesList.txt.train")
write_txt(off_train, "hateMemesList.txt.train")
write_txt(not_val, "goodMemesList.txt.val")
write_txt(off_val, "hateMemesList.txt.val")

In [ ]:
label_1 = df.copy()[df["label"] == 1]
label_0 = df.copy()[df["label"] == 0].sample(n=3300)
balanced = pd.concat([label_1, label_0], ignore_index=True).to_csv("../data/data/balanced_data.csv", index=False)


In [ ]:
balanced_df = pd.read_csv("../data/data/balanced_data.csv")

In [ ]:
bal_not_val = balanced_df[(balanced_df['set'] == 'val') & (balanced_df['label'] == 0)].reset_index(drop=True)
bal_off_val = balanced_df[(balanced_df['set'] == 'val') & (balanced_df['label'] == 1)].reset_index(drop=True)

In [ ]:
bal_not_train = balanced_df[(balanced_df['set'] == 'train') & (balanced_df['label'] == 0)].reset_index(drop=True)
bal_off_train = balanced_df[(balanced_df['set'] == 'train') & (balanced_df['label'] == 1)].reset_index(drop=True)

In [ ]:
bal_not_test = balanced_df[(balanced_df['set'] == 'test') & (balanced_df['label'] == 0)].reset_index(drop=True)
bal_off_test = balanced_df[(balanced_df['set'] == 'test') & (balanced_df['label'] == 1)].reset_index(drop=True)

In [ ]:
pd.concat([bal_not_test, bal_off_test])

In [ ]:
write_txt(bal_not_test, "bal.goodMemesList.txt.test")
write_txt(bal_off_test, "bal.hateMemesList.txt.test")
write_txt(bal_not_train, "bal.goodMemesList.txt.train")
write_txt(bal_off_train, "bal.hateMemesList.txt.train")
write_txt(bal_not_val, "bal.goodMemesList.txt.val")
write_txt(bal_off_val, "bal.hateMemesList.txt.val")

In [ ]:
def write_caption_to_file(df, basedir):
    for img, text in zip(df.loc[:, "img"].to_list(), df.loc[:, "text"].to_list()):
        filename = basedir + img + ".ocr"
        with open(filename, "w") as f:
            #print(filename)
            
            f.write(text)
        #break
    print("Done!")
    

In [ ]:
os.getcwd()

In [ ]:
write_caption_to_file(off_test, "/home/gushansad@GU.GU.SE/lt2318-ai/aics-project/data/data/")
write_caption_to_file(not_test, "/home/gushansad@GU.GU.SE/lt2318-ai/aics-project/data/data/")
write_caption_to_file(not_train, "/home/gushansad@GU.GU.SE/lt2318-ai/aics-project/data/data/")
write_caption_to_file(off_train, "/home/gushansad@GU.GU.SE/lt2318-ai/aics-project/data/data/")
write_caption_to_file(not_val, "/home/gushansad@GU.GU.SE/lt2318-ai/aics-project/data/data/")
write_caption_to_file(off_val, "/home/gushansad@GU.GU.SE/lt2318-ai/aics-project/data/data/")

In [ ]:
imgdir = os.listdir("../data/data/img")

In [ ]:
len([x for x in imgdir if x.endswith(".ocr")])

In [ ]:
# 0 = not offensive, 1 = offensive

In [ ]:
!ls ..

In [ ]:
list(open("../data/data/bal.hateMemesList.txt.val"))
list(open("../data/data/bal.hateMemesList.txt.val"))

In [ ]:
9000*0.7

In [ ]:
base = "../../data/img/"
for img in dev.img.to_list():
    old_path = base + img
    new_path = base + "dev/" + img
    os.rename(old_path, new_path)

In [ ]:
os.listdir("../../data/img/")

In [ ]:
love_paths = list(open(os.getcwd() + "/" + "garbage.txt"))

In [ ]:
love_paths

In [ ]:
os.getcwd()